### Imports
 Primeiramente são importadas as bibliotecass
nescessárias, sendo elas:
 - pandas: para leitura dos dados da base de dados
 - sklearn model_selection: para seleção dos modelos
 
 
    As demais são os métodos de regressão escolhidos:
 - Floresta Aleatória (Regressão)

In [268]:
import pandas as pd
import numpy as np
np.set_printoptions(legacy='1.25')

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import model_selection
from sklearn.model_selection import KFold

import optuna
from sklearn.model_selection import cross_val_score


### Base de Dados
Então realiza-se a leitura da base de dados contida no arquivo 'Analise Geral Normalizada.xlsx' e seus dados são divididos entre:
- X : dados de entrada da função.
- Y : valor a ser predito

In [269]:
df = pd.read_excel("Analise Geral Normalizada.xlsx")
array = df.values
X = array[:, :22]
y = array[:,22]
df.head()

,Vel. Vento (m/s) PoA,Dir. Vento (m/s) PoA,Chuva (mm) PoA,Nível adotado (cm) Guaiba,Chuva Taquari (cm),Nivel Taquari (m),Vazão Taquari (Mm3/s),Chuva Jacui (mm),Nivel Jacui (m),Vazão Jacui (Mm3/s),...,Chuva (cm) soma 8h Rio Pardo,Vel. Vento (m/s) média 8h Soledade,Dir. Vento (m/s) média 8h Soledade,Raj. Vento (m/s) média 8h Soledade,Chuva (cm) soma 8h Soledade,Vel. Vento (m/s) média 8h Bento,Dir. Vento (m/s) média 8h Bento,Raj. Vento (m/s) média 8h Bento,Chuva (cm) soma 8h Bento,Nível após uma semana (m)
0,0.2258,0.8889,0.0,0.2392,0.0,0.1499,0.0819,0.0,0.3241,0.2084,...,0.0,0.0402,0.7883,0.1456,0.0,0.0668,0.6186,0.0349,0.0,0.1478
1,0.1075,0.7500,0.0,0.2523,0.0,0.1534,0.0845,0.0,0.3449,0.2249,...,0.0,0.0013,0.4181,0.0366,0.0,0.4480,0.1275,0.3867,0.0,0.1425
2,0.1075,0.7500,0.0,0.2704,0.0,0.1618,0.0909,0.0,0.3554,0.2333,...,0.0,0.0000,0.2016,0.0000,0.0,0.3193,0.0898,0.1940,0.0,0.1747
3,0.3333,0.8889,0.0,0.2913,0.0,0.1704,0.0977,0.0,0.3658,0.2417,...,0.0,0.0665,0.8504,0.2277,0.0,0.1782,0.7920,0.1819,0.0,0.1934
4,0.3333,0.7500,0.0,0.3198,0.0,0.1821,0.1071,0.0,0.3898,0.2616,...,0.0,0.0414,0.4458,0.0952,0.0,0.0718,0.4771,0.0265,0.0,0.1553


TREINAMENTO

In [270]:
boosting = GradientBoostingRegressor(random_state=2024)

In [271]:
kfold = KFold(n_splits=5, shuffle=False)

In [272]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=2024)

### Analise inicial
São feitos alguns testes para analisar o funcionamento inicial da arvore com os dados disponiveis

In [273]:
boosting.fit(X_train, y_train)
boosting.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 2024,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [274]:
y_pred = boosting.predict(X_test)

In [275]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [276]:
mean_absolute_error(y_test, y_pred)

0.04002764400473383

In [277]:
mean_squared_error(y_test, y_pred)

0.002864653262708569

In [278]:
r2_score(y_test, y_pred)

0.9655253706219714

Pode se perceber que a floresta já possui saídas bastante satisfátórias com um erro quádratico médio bastante baixo e um score r2 bastante alto, portanto encontrar definições de hiperparametros melhores pode ser dificil

### OPTUNA e HIPERPARAMETROS
Agora são feitos testes com o frame optuna para descobrir quais os melhores hiperparametros para a função, sendo que os hiperparametros analisados foram:

- n_estimators: Número de árvores na floresta.
- max_depth: Tamanho máximo das arvores individuais.
- min_smaples_split: Número mínimo de amostras necessárias para dividir um nó.
- min_samples_leaf: Número mínimo de amostras necessárias em um nó folha.


In [279]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 0.1, 1)
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.1, 0.6)
    max_depth = trial.suggest_int('max_depth', 25, 45)
    min_samples_split = trial.suggest_int('min_samples_split', 10, 50)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    model = GradientBoostingRegressor(n_estimators=n_estimators,
    learning_rate=learning_rate,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    alpha=alpha,
    min_samples_leaf=min_samples_leaf)

    score = cross_val_score(model, X, y, n_jobs=-1, cv=kfold, scoring='neg_mean_squared_error').mean()

    return score

In [280]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=2024))

[I 2024-08-18 12:21:45,923] A new study created in memory with name: no-name-0c4e53c2-e74f-448d-81b0-57a4c290975d


In [281]:
study.optimize(objective, n_trials=400)

[I 2024-08-18 12:21:47,073] Trial 0 finished with value: -0.02963204116427771 and parameters: {'alpha': 0.6292130670058581, 'n_estimators': 155, 'learning_rate': 0.194075980019253, 'max_depth': 25, 'min_samples_split': 18, 'min_samples_leaf': 3}. Best is trial 0 with value: -0.02963204116427771.
[I 2024-08-18 12:21:47,605] Trial 1 finished with value: -0.024782607350452773 and parameters: {'alpha': 0.7545161293160093, 'n_estimators': 152, 'learning_rate': 0.3369228517041093, 'max_depth': 34, 'min_samples_split': 10, 'min_samples_leaf': 16}. Best is trial 1 with value: -0.024782607350452773.
[I 2024-08-18 12:21:48,686] Trial 2 finished with value: -0.023129320751057107 and parameters: {'alpha': 0.6422036851073138, 'n_estimators': 195, 'learning_rate': 0.4321843236782378, 'max_depth': 37, 'min_samples_split': 28, 'min_samples_leaf': 5}. Best is trial 2 with value: -0.023129320751057107.
[I 2024-08-18 12:21:49,322] Trial 3 finished with value: -0.02262235364572334 and parameters: {'alpha'

In [282]:

best_params = study.best_params
best_score = study.best_value
print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_score:.4f}")

Best Hyperparameters: {'alpha': 0.9748292519846857, 'n_estimators': 94, 'learning_rate': 0.4335532438354215, 'max_depth': 36, 'min_samples_split': 45, 'min_samples_leaf': 1}
Best Accuracy: -0.0204


In [283]:
optuna.visualization.plot_optimization_history(study)

In [284]:
optuna.visualization.plot_slice(study, params=['n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split', 'learning_rate', 'alpha'])

Aqui pode se perceber que min_samples_leaf e min_samples_split possuem uma influencia maior sobre a saída sendo valores mais baixos de ambos melhores, pois podemos ver uma certa ordem nas saidas, enquanto max_depth e n_estimators praticamente não possuem ordem

In [285]:
optuna.visualization.plot_param_importances(study)

Expresso aqui também a importancia de min_samples_leaf e min_samples_split

### AVALIAÇÃO FINAL

In [286]:
best_n_estimators = best_params['n_estimators']
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']
best_min_samples_leaf = best_params['min_samples_leaf']
best_learning_rate = best_params['learning_rate']


boostingFinal = GradientBoostingRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth, min_samples_split=best_min_samples_split, min_samples_leaf=best_min_samples_leaf, learning_rate=best_learning_rate)


In [287]:
boostingFinal.fit(X_train, y_train)
y_pred = boostingFinal.predict(X_test)

In [288]:

mean_absolute_error(y_test, y_pred)


0.03831492161069277

In [289]:
mean_squared_error(y_test, y_pred)

0.0020766254519997826

In [290]:

r2_score(y_test, y_pred)

0.9750088802206437